# Purpose: Convert SST-1 Sample Sheet Metadata to MediaWiki Table

## Imports and File Selection 

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import date
import openpyxl

# Import  Spreadsheet Reformat function
from rsoxs_scans.spreadsheets import convertSampleSheetExcelMediaWiki

In [4]:
# Provide path to excel sheet
# Ex: r'/nsls2/users/bpatel/Sample_Bar_template_v2022_11_beta_2_BPSuggestion.xlsx'
excelSheetInputPath = Path("../Sample_Bar_v2023_2.xlsx"
)

## Function to Load and Process Data

In [5]:
def convertSampleSheetExcelMediaWiki(
    excelSheet: Path = None,
    paramsSheetToOutput: str = "all",
    rulesSheetName: str = "SheetRulesAndMetaData",
    versionCell: str = "B4",
    startRow_Params: int = 7,
    endRow_Params: int = None,
    startColumn_Params: str = "A",
    endColumn_Params: str = "F",
    verbose: bool = "TRUE",
) -> str:
    """Converts Sample Sheet Parameter Metadata into a MediaWiki-compatible format string.

    Parameters
    ----------
    excelSheet: Path
        Path (or string) to the excel sheet to be loaded.
    rulesSheetName: str
        Name of the excel sheet which should be parsed for metadata
    paramsSheetToOutput: str
        Which set of params should be output (e.g., 'bar', 'acquisitions'). 'all' will sequentially output tables for the same wiki page
    versionCell: str
        Location (e.g., 'B4') of the cell that contains the sheet version number
    startRow_Params: int
        Excel row number which contains the header for the metadata table (excel starts at row 1)
    endRow_Params: int
        Excel row number which contains the last row of metadata (leave as -1 if scanning to end of file)
    startColumn_Params: str
        First excel column (by letter) that contains the metadata table
    endColumn_Params:str
        Last excel column (by letter) that contains the metadata table
    verbose: bool
        Whether to print progress text to stdout


    Returns
    -------
    str
        A string containing the formatted table ready to copy-paste into MediaWiki
    """

    if verbose:
        print("-" * 5 + " Start Log" + "-" * 5)
        print("\tExtracting Sheet Version...", end=" ")

    # Step 1: Extract Version Cell and add to wiki table header
    ## Split versionCell in (row, column)
    versRow, versColumn = [
        int("".join(filter(str.isdigit, versionCell))),
        "".join(filter(str.isalpha, versionCell)),
    ]

    ## Extract Version Code as a string
    versionStr = openpyxl.load_workbook(excelSheet).properties.title
    # print(versionStr)

    ## Get Current Date
    date.today()

    ## Add Wiki Page Header to Output string
    outStr = (
        f"== SST-1 Sample Sheet Syntax Version: {versionStr} Last Updated: {date.today()} ==\n"
    )
    if verbose:
        print(f"Pass!\n\t\tVersion Number is -> {versionStr}")

    # Step 2: Extract Metadata Table from Excel Sheet
    if verbose:
        print("\tExtracting Sheet Metadata...")

    ## If endRow_Params is provided, limit the number of rows parsed
    if endRow_Params is None:
        numRows = None
    else:
        numRows = endRow_Params - startRow_Params

    ## Convert column bounds to string
    colString = startColumn_Params + ":" + endColumn_Params

    excelMetadataIn = pd.read_excel(
        excelSheet,
        sheet_name=rulesSheetName,
        header=startRow_Params - 1,
        nrows=numRows,
        usecols=colString,
    )

    ## Drop empty rows (where 'Sheet' is NaN)
    excelMetadataIn = excelMetadataIn.dropna(subset="Sheet")
    ## Replace NaNs and 'nan's with blank
    excelMetadataIn = excelMetadataIn.replace("nan", "")
    excelMetadataIn = excelMetadataIn.fillna(" ")

    ###display(excelMetadataIn)

    ## Build MediaWiki Tables

    ## Get list of unique sheets for which we have metadata
    sheetList = excelMetadataIn.Sheet.unique()

    if verbose:
        print(f"\t\tFound Metadata for these sheets: {sheetList}")
        print(f"\t\tUser requested tables for: {paramsSheetToOutput}")

    # Filter down the list of tables to output
    if paramsSheetToOutput.lower() == "all":
        sheetListToRun = sheetList
    else:
        sheetListToRun = [paramsSheetToOutput]

    if verbose:
        print(f"\t\tOutputting tables for: {sheetListToRun}...")

    # Create subset dataframes
    dataframeList = []
    for sheetName in sheetListToRun:
        dataframeList.append(excelMetadataIn[excelMetadataIn.Sheet == sheetName])

    ###print(dataframeList[1])

    ## Make one table per value in the 'Sheet' column
    for excelMetadataFrame in dataframeList:
        excelMetadataFrame.reset_index(drop=True, inplace=True)
        outStr += "\n" + r'{| class="wikitable sortable"' + "\n" + "|-\n"

        # Add header row elements
        outStr += "! "
        for colHeader in excelMetadataFrame.columns:
            filteredColHeader = str(colHeader).replace("\r", " ").replace("\n", " ")
            outStr += f"{filteredColHeader} !! "

        # trim extra "!! "
        outStr = outStr[:-4]

        ###display(excelMetadataFrame)

        # Add Metadata Row Elements

        ## Loop through metadata rows
        for mdRow in excelMetadataFrame.index:
            ###Loop through columns
            outStr += "\n|-\n| "
            ###print(mdRow)
            for mdVal in excelMetadataFrame.iloc[mdRow].to_list():
                filteredmdVal = str(mdVal).replace("\r", " ").replace("\n", " ")
                outStr += f"{filteredmdVal} || "
                ###print(f"\t{mdVal}")
            # trim extra " || " at the end of each line
            outStr = outStr[:-4]

        # Add MediaWiki Table End
        outStr += "\n|}\n"
        # print(outStr)

    if verbose:
        print("-" * 5 + " End Log. Copy text below this line into the wiki" + "-" * 5)

    return outStr

## Example Usage: All Tables in one Wiki Page

https://wiki-nsls2.bnl.gov/beamline7ID1/index.php?title=User:Bijal

In [6]:
print(convertSampleSheetExcelMediaWiki(excelSheetInputPath, verbose=False))

== SST-1 Sample Sheet Syntax Version: 2023-2 Version 1.0 Last Updated: 2023-08-09 ==

{| class="wikitable sortable"
|-
! Sheet !! Parameter !! Description !! Rules !! Example !! Notes
|-
| Bar || bar_name || REQUIRED: Unique name for this bar || Type: String All rows in the sheet must contain the same value for bar_name. || testbar ||  
|-
| Bar || sample_id || REQUIRED: Unique identifier for the sample.  || Type: String Must match any physical sample labels. All rows must have unique sample_id. || EG01 || This is what will be referenced in the acquisitions, so keep it reasonably short. Recommend to stick to letter,numbers, and underscore.
|-
| Bar || sample_name || REQUIRED: Identifier for the sample (can be non-unique) || Type: String Plain english is encouraged || P3HT-AN-120C || Will be in file name (readable by whoever is loading and measuring samples)
|-
| Bar || project_name || REQUIRED: Used as the name of the output data folder. || Type: String Any characters allowed on a linu

## Example Usage: Individual Tables for separate Wiki Pages

In [179]:
# Example Output see: https://wiki-nsls2.bnl.gov/beamline7ID1/index.php?title=User:Bijal

# Bar only
#print(convertSampleSheetExcelMediaWiki(excelSheetInputPath, verbose=True, paramsSheetToOutput='Bar'))

# Acquisitions only
print(convertSampleSheetExcelMediaWiki(excelSheetInputPath, verbose=False, paramsSheetToOutput='Acquisitions'))

== SST-1 Sample Sheet Syntax Version: 2023-1.1 Last Updated: 2023-01-06 ==

{| class="wikitable sortable"
|-
! Sheet !! Parameter !! Description !! Rules !! Example !! Notes
|-
| Acquisitions || sample_id || Must exactly match a sample_id from the Bar sheet ||   ||   ||  
|-
| Acquisitions || configuration || Measurement Configuration || Choose from WAXSNEXAFS, WAXS, SAXS, SAXSNEXAFS, SAXS_liquid, WAXS_liquid || SAXS || Determines which detector is used / slits
|-
| Acquisitions || type || Type of measurement || Choose from RSoXS, NEXAFS, Spiral || Spiral || Depending on the value of this cell, unnecessary acquisition parameters will be greyed out and locked. 
|-
| Acquisitions || priority || Determines which order scans will be run, lowest value first. || Must be an integer from 1 to 100 || 2 || Normal way to sort queue steps
|-
| Acquisitions || edge || Which elemental edge or energy ranges you wish to scan. || Must match an entry in the lookup table (see Notes), OR be a single energ

# Test Code during development of this function. 

In [14]:
def convertSampleSheetExcelMediaWiki2(
    excelSheet: Path = None,
    paramsSheetToOutput: str = "all",
    rulesSheetName: str = "SheetRulesAndMetaData",
    versionCell: str = "B4",
    startRow_Params: int = 7,
    endRow_Params: int = None,
    startColumn_Params: str = "A",
    endColumn_Params: str = "F",
) -> str:
    """Converts Sample Sheet Parameter Metadata into a MediaWiki-compatible format string.

    Parameters
    ----------
    excelSheet: Path
        Path (or string) to the excel sheet to be loaded.
    rulesSheetName: str
        Name of the excel sheet which should be parsed for metadata
    paramsSheetToOutput: str
        Which set of params should be output (e.g., 'bar', 'acquisitions'). 'all' will sequentially output tables for each sheet
    versionCell: str
        Location (e.g., 'B4') of the cell that contains the sheet version number
    startRow_Params: int
        Excel row number which contains the header for the metadata table (excel starts at row 1)
    endRow_Params: int
        Excel row number which contains the last row of metadata (leave as -1 if scanning to end of file)
    startColumn_Params: str
        First excel column (by letter) that contains the metadata table
    endColumn_Params:str
        Last excel column (by letter) that contains the metadata table


    Returns
    -------
    str
        A string containing the formatted table ready to copy-paste into MediaWiki
    """
    # Split versionCell in (row, column)
    versRow, versColumn = [
        int("".join(filter(str.isdigit, versionCell))),
        "".join(filter(str.isalpha, versionCell)),
    ]

    # Extract Version Code as a string
    versionStr = pd.read_excel(
        excelSheet,
        sheet_name=rulesSheetName,
        index_col=None,
        usecols=versColumn,
        nrows=0,
        header=versRow - 1,
    )
    versionStr = versionStr.columns.values[0]
    # print(versionStr)

    # Get Current Date
    date.today()

    # Add Wiki Page Header to Output string
    outStr = (
        f"== SST-1 Sample Sheet Syntax Version: {versionStr} Last Updated: {date.today()} ==\n"
    )

    # Extract Metadata Table

    # If endRow_Params is provided, limit the number of rows parsed
    if endRow_Params is None:
        numRows = None
    else:
        numRows = endRow_Params - startRow_Params

    # Convert column bounds to string
    colString = startColumn_Params + ":" + endColumn_Params

    excelMetadataIn = pd.read_excel(
        excelSheet,
        sheet_name=rulesSheetName,
        header=startRow_Params - 1,
        nrows=numRows,
        usecols=colString,
    )

    # Replace NaNs and 'nan's with blank
    excelMetadataIn = excelMetadataIn.replace("nan", "")
    excelMetadataIn = excelMetadataIn.fillna("")
    # display(excelMetadataIn)

    # Construct MediaWiki Table
    outStr += "\n" + r'{| class="wikitable sortable"' + "\n" + "|-\n"

    # Add header row elements
    outStr += "! "
    for colHeader in excelMetadataIn.columns:
        filteredColHeader = str(colHeader).replace("\r", " ").replace("\n", " ")
        outStr += f"{filteredColHeader} !! "

    # trim extra "!! "
    outStr = outStr[:-4]

    # Add Metadata Row Elements

    ## Loop through metadata rows
    for mdRow in excelMetadataIn.index - 1:
        ###Loop through columns
        outStr += "\n|-\n| "
        for mdVal in excelMetadataIn.iloc[mdRow].to_list():
            filteredmdVal = str(mdVal).replace("\r", " ").replace("\n", " ")
            outStr += f"{filteredmdVal} || "
            pass
            # print(str(mdVal) + "\n")

    # Add MediaWiki Table End
    outStr += "\n|}"
    # print(outStr)

    return outStr

In [15]:
excelSheet = excelSheetInputPath
rulesSheetName = "SheetRulesAndMetaData"
versionCell = "B4"
startRow_Params = 7
endRow_Params = None
startColumn_Params = "A"
endColumn_Params = "F"

In [16]:
# Split versionCell in (row, column)
versRow, versColumn = [
    int("".join(filter(str.isdigit, versionCell))),
    "".join(filter(str.isalpha, versionCell)),
]

# Extract Version Code as a string
versionStr = pd.read_excel(
    excelSheet,
    sheet_name=rulesSheetName,
    index_col=None,
    usecols=versColumn,
    nrows=0,
    header=versRow - 1,
)
versionStr = versionStr.columns.values[0]
# print(versionStr)

ValueError: Passed header=[3], len of 1, but only 1 lines in file (sheet: SheetRulesAndMetaData)

In [17]:
# Extract Metadata Table

# If endRow_Params is provided, limit the number of rows parsed
if endRow_Params is None:
    numRows = None
else:
    numRows = endRow_Params - startRow_Params

# Convert column bounds to string
colString = startColumn_Params + ":" + endColumn_Params

excelMetadataIn = pd.read_excel(
    excelSheet,
    sheet_name=rulesSheetName,
    header=startRow_Params - 1,
    nrows=numRows,
    usecols=colString,
)

# Replace NaNs and 'nan's with blank
excelMetadataIn = excelMetadataIn.replace("nan", "")
excelMetadataIn = excelMetadataIn.fillna("")
# display(excelMetadataIn)

# Construct MediaWiki Table
outStr = r'{| class="wikitable sortable"' + "\n" + "|-\n"

# Add header row elements
outStr += "! "
for colHeader in excelMetadataIn.columns:
    filteredColHeader = str(colHeader).replace("\r", " ").replace("\n", " ")
    outStr += f"{filteredColHeader} !! "

# trim extra "!! "
outStr = outStr[:-4]

# Add Metadata Row Elements

## Loop through metadata rows
for mdRow in excelMetadataIn.index - 1:
    ###Loop through columns
    outStr += "\n|-\n| "
    for mdVal in excelMetadataIn.iloc[mdRow].to_list():
        filteredmdVal = str(mdVal).replace("\r", " ").replace("\n", " ")
        outStr += f"{filteredmdVal} || "
        pass
        # print(str(mdVal) + "\n")

# Add MediaWiki Table End
outStr += "\n|}"
# print(outStr)

return outStr

SyntaxError: 'return' outside function (1516084869.py, line 53)

In [ ]:
excelDataIn